# Moving To Richmond

In [ ]:
##!conda install -c anaconda beautifulsoup4 -y
##!conda install -c anaconda lxml -y
##!conda install -c anaconda requests -y
##!conda config --add channels conda-forge
##!conda install -c conda-forge geopy --yes
##!conda install -c conda-forge folium=0.5.0 --yes
##  imports and functions
from project_lib import Project
from bs4 import BeautifulSoup
import requests
import lxml
import csv
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from geopy import distance
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML     
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns
# import k-means from clustering stage
from sklearn.cluster import KMeans

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
RVA_Cities =['richmond, va','chesterfield, va', 'midlothian, va', 'henrico, va', 'glen allen, va', 'ashland, va','chmamberlayne, va', 'mechanicsville, va']
new_job_addr = '7100 Forest Ave 23226'
rva_gs = ['Grocery Store', 'Supermarket','Drugstore', 'Pharmacy', 'Market', 'Coffee Shop', 'Wine Shop','Gym / Fitness Center','Salon / Barbershop','Gym','Farmers Market', 'Food & Drink Shop', 'Performing Arts Venue','Museum', 'Theater','Art Museum','Dog Run','Social Club','State / Provincial Park', 'Lake']


In [ ]:
zipfile = pd.read_csv("https://move-econresearch-prod.s3-us-west-2.amazonaws.com/listings/core/current_month/RDC_Inventory_Core_Metrics_Zip_Current.csv")

In [ ]:
zip2 = zipfile[['postal_code','zip_name', 'median_listing_price','average_listing_price','new_listing_count','active_listing_count','total_listing_count','price_reduced_count','median_days_on_market']]
##zip2.head()
rva_df = zip2.loc[zip2['zip_name'].isin(RVA_Cities)]
rva_df.rename(columns={'postal_code': 'ZipCode'},inplace=True)
rva_df

In [ ]:
##Use geopy library to get the latitude and longitude values of new jobs address
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(new_job_addr)
nja_latitude = location.latitude
nja_longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(new_job_addr, nja_latitude, nja_longitude))
print(location.raw['display_name'])

In [ ]:
coords = []
 
geolocator = Nominatim(user_agent="my-application")
for zipcode, city in zip(rva_df['ZipCode'],rva_df['zip_name'] ):
    location = geolocator.geocode({"postalcode": zipcode, "state": 'VA'})
    latitude2 = location.latitude
    longitude2 = location.longitude
    ##print(location)
    miles = distance.distance((nja_latitude,nja_longitude),(latitude2,longitude2)).mi
    #print(round(miles))
    coords.append((
        zipcode,
        latitude2,
        longitude2,
        miles,
        location.raw['display_name']))

In [ ]:
# dconvert the list into a new DataFrame
coords_df = pd.DataFrame(coords)
#define the column names
coords_df.columns = ['ZipCode','Latitude', 'Longitude', 'Distance','display_name']

print(coords_df.shape)
coords_df.head()

In [ ]:
mylist = coords_df['display_name'].to_list()
cleaner = []
for row in mylist:
    x = row.split(',')
    cleaner.append((x.pop(0)))
coords_df['display_name'] = pd.Series(cleaner)
coords_df.head()

In [ ]:
# create map
map_zip = folium.Map(location=[nja_latitude, nja_longitude], zoom_start=12)

label = '7100 Forest Ave 23226'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [nja_latitude, nja_longitude],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#FDBB84',
    fill_opacity=0.7).add_to(map_zip)  

 
for lat, lng, zipcode, dispname in zip(coords_df['Latitude'], coords_df['Longitude'],coords_df['ZipCode'], coords_df['display_name']):
    label = '{}, {}'.format(dispname, zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_zip)        
map_zip

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
##get the area info for goods and services
## Foursquare API required inputs
VERSION = '20180604'
LIMIT = 100
radius = 5000
venue_list = []

##loop thru and get info from 4square
for lat, lng, zipcode, dispname in zip(coords_df['Latitude'], coords_df['Longitude'],coords_df['ZipCode'], coords_df['display_name']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()

    items = results['response']['groups'][0]['items']
    ##print(items)
    ##break
    for venue in items:
        try:
            venue_list.append(( 
                zipcode,
                dispname, 
                venue['venue']['name'], 
                venue['venue']['location']['address'],
                venue['venue']['categories'][0]['name']))
        except:
            continue

In [ ]:
# dconvert the venue list into a new DataFrame
venues_df = pd.DataFrame(venue_list)
#define the column names
venues_df.columns = ['ZipCode','Display_Name', 'VenueName', 'VenueAddress', 'Category']

print(venues_df.shape)


In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['Category'].unique())))

In [ ]:
##reduce the number of categories to the ones of interest
rva_ven = venues_df.loc[venues_df['Category'].isin(rva_gs)]
##rva_ven.head()

In [ ]:
###group everything up and see what is where
rva_onehot = pd.get_dummies(rva_ven[['Category']], prefix="", prefix_sep="")

#print one hot totals
print( rva_onehot.sum(axis = 0, skipna = True))

# add zipcode and name column back to dataframe
 
rva_onehot['ZipCode'] = rva_ven['ZipCode'] 


#  zipcode and displayname column to the first column
fixed_columns = list(rva_onehot.columns[-2:]) + list(rva_onehot.columns[:-2])
rva_onehot = rva_onehot[fixed_columns]

print("OneHot Shape: {}".format(rva_onehot.shape))
rva_onehot.head()

rva_grouped = rva_onehot.groupby(['ZipCode']).sum().reset_index()

print("Grouped Shape: {}".format(rva_grouped.shape))
print(rva_grouped)

In [ ]:
##of the venues of interest what are we most likely to find in the zipcodea

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['ZipCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['ZipCode'] = rva_grouped['ZipCode']

for ind in np.arange(rva_grouped.shape[0]):
    postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(rva_grouped.iloc[ind, :], num_top_venues)

postcode_venues_sorted

In [ ]:
## compare to most common place of all places
rva_onehot2 = pd.get_dummies(venues_df[['Category']], prefix="", prefix_sep="")

# add zipcode and displayname column back to dataframe
 
rva_onehot2['ZipCode'] = venues_df['ZipCode'] 
rva_onehot2['Display_Name'] = venues_df['Display_Name']

#  zipcode and displayname column to the first column
fixed_columns = list(rva_onehot2.columns[-2:]) + list(rva_onehot2.columns[:-2])
rva_onehot2 = rva_onehot2[fixed_columns]

print(rva_onehot2.shape)
rva_onehot2.head()

rva_grouped2 = rva_onehot2.groupby(['ZipCode']).sum().reset_index()

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['ZipCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['ZipCode'] = rva_grouped2['ZipCode']

for ind in np.arange(rva_grouped2.shape[0]):
    postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(rva_grouped2.iloc[ind, :], num_top_venues)

postcode_venues_sorted

In [ ]:
#Now lets look at the real estate 
rva_df.corr()

In [ ]:
sns.regplot(x="median_listing_price", y="median_days_on_market", data=rva_df)

In [ ]:
sns.regplot(x="price_reduced_count", y="median_days_on_market", data=rva_df)

In [ ]:
sns.regplot(x="price_reduced_count", y="active_listing_count", data=rva_df)

In [ ]:
rva_merged = pd.merge(rva_df, rva_grouped, how='left',
        on='ZipCode', validate="1:1")
rva_merged.head()
##write results to csv to import into excel or other uses for later
project.save_data(file_name = "merged_data.csv",data = rva_merged.to_csv(index=False))


In [ ]:
rva_merged.corr()

In [ ]:
sns.regplot(x="average_listing_price", y="Coffee Shop", data=rva_merged)